In [3]:
import numpy as np
from sklearn.svm import SVC
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, hamming_loss
from sklearn import preprocessing
from sklearn.svm import LinearSVC
from imblearn.over_sampling import SMOTE



In [ ]:
x=np.array([[0,0],[1,0],[0,-1],[-1,0]])
y=np.array([-1,1,1,1])
clf=SVC(C=10000,kernel="poly",degree=2,gamma=1,coef0=1)
clf.fit(x,y).dual_coef_

6(a):Choose 70% of the data ran- domly as the training set.

In [26]:
data=pd.read_csv("../data/Anuran Calls (MFCCs)/Frogs_MFCCs.csv")
label=data.iloc[:,22:26]
print(label)
data=data.iloc[:,:22]
print(data)
data_train, data_test,y_train, y_test=train_test_split(data,label, test_size=0.3, random_state=42)
print(np.shape(data_train))
print(np.shape(data_test))
le = preprocessing.LabelEncoder()
# Apply SMOTE to the training set
smote = SMOTE(random_state=42)
summary_param=pd.DataFrame()


               Family      Genus         Species  RecordID
0     Leptodactylidae  Adenomera  AdenomeraAndre         1
1     Leptodactylidae  Adenomera  AdenomeraAndre         1
2     Leptodactylidae  Adenomera  AdenomeraAndre         1
3     Leptodactylidae  Adenomera  AdenomeraAndre         1
4     Leptodactylidae  Adenomera  AdenomeraAndre         1
...               ...        ...             ...       ...
7190          Hylidae     Scinax     ScinaxRuber        60
7191          Hylidae     Scinax     ScinaxRuber        60
7192          Hylidae     Scinax     ScinaxRuber        60
7193          Hylidae     Scinax     ScinaxRuber        60
7194          Hylidae     Scinax     ScinaxRuber        60

[7195 rows x 4 columns]
      MFCCs_ 1  MFCCs_ 2  MFCCs_ 3  MFCCs_ 4  MFCCs_ 5  MFCCs_ 6  MFCCs_ 7  \
0          1.0  0.152936 -0.105586  0.200722  0.317201  0.260764  0.100945   
1          1.0  0.171534 -0.098975  0.268425  0.338672  0.268353  0.060835   
2          1.0  0.152317 -0.08297

In [16]:
#SVM for each of the labels, using Gaussian kernels and one versus all classifiers
#input i,j:the weight of the SVM penalty
#input cv :cross validation
#input y :target y label
def svm_fixed_gamma(i,j,cv,y):
    gamma=np.arange(0.1, 2.1, 0.1)
    param_grid = {'C': [i,j],"gamma":gamma}
    # Create the SVM classifier
    svm = SVC(kernel="rbf")
    # Perform grid search with 10-fold cross-validation to find the best hyperparameters
    grid_search = GridSearchCV(svm, param_grid, scoring=make_scorer(hamming_loss), cv=cv)
    grid_search.fit(data_train, le.fit_transform(y))
    return grid_search.cv_results_["mean_test_score"],grid_search.cv_results_
#l1 penalized SVM
#input i,j:the weight of the SVM penalty
#input cv :cross validation
#input y :target y label
def svm_l1(i,j,cv,y):
    
    param_grid = {'C': [i,j]}
    # Create the SVM classifier
    svm = LinearSVC(penalty='l1', dual=False, random_state=42)
    # Perform grid search with 10-fold cross-validation to find the best hyperparameters
    grid_search = GridSearchCV(svm, param_grid,scoring=make_scorer(hamming_loss), cv=cv)
    grid_search.fit(data_train, le.fit_transform(y))
    return grid_search.cv_results_["mean_test_score"],grid_search.cv_results_

#l1 penalized SVM smote
#input i,j:the weight of the SVM penalty
#input cv :cross validation
#input y :target y label
def svm_l1_smote(i,j,cv,y):
    param_grid = {'C': [i,j]}
    # Create the SVM classifier
    svm = LinearSVC(penalty='l1', dual=False, random_state=42,max_iter=1000)
    # Perform grid search with 10-fold cross-validation to find the best hyperparameters
    grid_search = GridSearchCV(svm, param_grid, scoring=make_scorer(hamming_loss), cv=cv)
    grid_search.fit(smote.fit_resample(data_train,le.fit_transform(y))[0],smote.fit_resample(data_train,le.fit_transform(y))[1])
    return grid_search.cv_results_["mean_test_score"],grid_search.cv_results_

#find the best parameter
def best_param(x,y):
    best_accuracy=1
    param=set()
    for i in range(1,6):
        accuracy=x(10**i,10**(-i),4,y)
        if best_accuracy>min(accuracy[0]):
            best_accuracy=min(accuracy[0])
            param=accuracy[1]["params"][np.argmin(min(accuracy[0]))]
    print(best_accuracy,param)
    
#evaluate classifier
def eva(c,y,y_test):
    svm = LinearSVC(penalty='l1', dual=False, random_state=42,C=c,max_iter=1000)
    svm.fit(data_train,le.fit_transform(y))
    return svm.score(data_test,le.fit_transform(y_test))

#evaluate classifier
def eva_Gaus(c,y,y_test,gamma):
    svm = SVC(kernel="rbf",C=c,gamma=gamma)
    svm.fit(data_train,le.fit_transform(y))
    return svm.score(data_test,le.fit_transform(y_test))
#evaluate classifier
def eva_smote(c,y,y_test):
    svm = LinearSVC(penalty='l1', dual=False, random_state=42,C=c,max_iter=1000)
    svm.fit(smote.fit_resample(data_train,le.fit_transform(y))[0],smote.fit_resample(data_train,le.fit_transform(y))[1])
    return svm.score(data_test,le.fit_transform(y_test))

0.006949960285941224 {'C': 10, 'gamma': 0.1}

In [27]:
summary_param["svm_fixed_gamma"]=[{'C': 10, 'gamma': 0.1},{'C': 10, 'gamma': 0.1},{'C': 10, 'gamma': 0.1}]
summary_param

,svm_fixed_gamma
0,"{'C': 10, 'gamma': 0.1}"
1,"{'C': 10, 'gamma': 0.1}"
2,"{'C': 10, 'gamma': 0.1}"


In [168]:
#the best parameter for Family label
best_param(svm_fixed_gamma,y_train.iloc[:,0])    

(array([0.03415409, 0.01945989, 0.0166799 , 0.01409849, 0.0127085 ,
       0.01092137, 0.01111994, 0.01012708, 0.00972994, 0.00873709,
       0.00833995, 0.00794281, 0.00794281, 0.00774424, 0.00794281,
       0.0073471 , 0.00714853, 0.00694996, 0.00694996, 0.00714853,
       0.15389198, 0.08439237, 0.06473392, 0.0581811 , 0.05361398,
       0.05043685, 0.04725973, 0.04606831, 0.04269261, 0.04090548,
       0.03852264, 0.03653693, 0.03455123, 0.03375695, 0.03256553,
       0.03097697, 0.03018268, 0.02978554, 0.02879269, 0.02859412]), {'mean_fit_time': array([0.12118995, 0.09127033, 0.0781495 , 0.0725143 , 0.06798208,
       0.0631538 , 0.06227762, 0.0601368 , 0.05895054, 0.06022239,
       0.05955493, 0.05899203, 0.05816704, 0.05784732, 0.0593912 ,
       0.05944639, 0.05921453, 0.05893373, 0.06021631, 0.06122476,
       0.35916066, 0.28913325, 0.25290239, 0.23115253, 0.20968115,
       0.20022154, 0.19030321, 0.17866629, 0.17149395, 0.16731399,
       0.16134673, 0.15840912, 0.15470445

0.008737092930897538 {'C': 10, 'gamma': 0.1}

In [170]:
#the best parameter for Genus label
best_param(svm_fixed_gamma,y_train.iloc[:,1])    

(array([0.02640985, 0.02025417, 0.01628276, 0.01330421, 0.01191422,
       0.01191422, 0.01211279, 0.01171565, 0.01092137, 0.01092137,
       0.0107228 , 0.0107228 , 0.01032566, 0.00992851, 0.00913423,
       0.00893566, 0.00873709, 0.00893566, 0.00873709, 0.00893566,
       0.19857029, 0.1316521 , 0.11219222, 0.10345512, 0.0895552 ,
       0.07982526, 0.0782367 , 0.07545671, 0.07188245, 0.06850675,
       0.06592534, 0.06493249, 0.0641382 , 0.06334392, 0.06294678,
       0.06155679, 0.06076251, 0.06076251, 0.05976966, 0.05957109]), {'mean_fit_time': array([0.10562354, 0.08085626, 0.07279283, 0.06763083, 0.06501698,
       0.06424367, 0.06463194, 0.06409109, 0.06381345, 0.06437135,
       0.06472075, 0.06717944, 0.06864184, 0.06978244, 0.07224864,
       0.07404917, 0.07848817, 0.07945508, 0.08153284, 0.08482367,
       0.36768556, 0.31550694, 0.27716786, 0.25208515, 0.23748761,
       0.22624606, 0.21955454, 0.20902324, 0.20060468, 0.19599158,
       0.19680905, 0.20573455, 0.18762243

0.009332803812549643 {'C': 10, 'gamma': 0.1}

In [171]:
#the best parameter for Species label
best_param(svm_fixed_gamma,y_train.iloc[:,2]) 

(array([0.02323272, 0.0166799 , 0.01449563, 0.01290707, 0.01290707,
       0.01171565, 0.01111994, 0.0107228 , 0.0107228 , 0.01032566,
       0.01092137, 0.01092137, 0.01092137, 0.01052423, 0.00992851,
       0.00992851, 0.00953137, 0.0093328 , 0.0093328 , 0.0093328 ,
       0.17255759, 0.12291501, 0.10365369, 0.08915806, 0.08101668,
       0.07525814, 0.07466243, 0.07148531, 0.06870532, 0.06731533,
       0.06354249, 0.06135822, 0.0601668 , 0.05877681, 0.05778396,
       0.05599682, 0.05520254, 0.05420969, 0.05381255, 0.05301827]), {'mean_fit_time': array([0.0635193 , 0.05305755, 0.05237865, 0.04907632, 0.04936182,
       0.04966623, 0.05093414, 0.05289358, 0.05450243, 0.05809104,
       0.05980402, 0.06187779, 0.06439924, 0.06725097, 0.06912386,
       0.07316625, 0.07398242, 0.07722086, 0.07965797, 0.0822739 ,
       0.34441549, 0.26509982, 0.23005748, 0.21061391, 0.20031625,
       0.18827903, 0.21253002, 0.2058568 , 0.1709426 , 0.17308158,
       0.17532903, 0.17147523, 0.16920626

0.08975377283558379 {'C': 10000}

In [28]:
summary_param["svm_l1"]=[{'C': 100},{'C': 1000},{'C': 10000}]

In [7]:
#the best parameter for Species label with l1-SVM
best_param(svm_l1,y_train.iloc[:,2]) 

/Users/huangjie/opt/anaconda3/envs/ee541_work/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/huangjie/opt/anaconda3/envs/ee541_work/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/huangjie/opt/anaconda3/envs/ee541_work/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/huangjie/opt/anaconda3/envs/ee541_work/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/huangjie/opt/anaconda3/envs/ee541_work/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  

0.08975377283558379 {'C': 10000}


/Users/huangjie/opt/anaconda3/envs/ee541_work/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


0.05877680698967434 {'C': 100}

In [182]:
#the best parameter for Family label with l1-SVM
best_param(svm_l1,y_train.iloc[:,0])  

/Users/huangjie/opt/anaconda3/envs/ee541_work/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/huangjie/opt/anaconda3/envs/ee541_work/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/huangjie/opt/anaconda3/envs/ee541_work/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/huangjie/opt/anaconda3/envs/ee541_work/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/huangjie/opt/anaconda3/envs/ee541_work/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  

0.05877680698967434 {'C': 100}


/Users/huangjie/opt/anaconda3/envs/ee541_work/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


0.048451151707704525 {'C': 1000}

In [9]:
#the best parameter for Genus label with l1-SVM
best_param(svm_l1,y_train.iloc[:,1]) 

/Users/huangjie/opt/anaconda3/envs/ee541_work/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/huangjie/opt/anaconda3/envs/ee541_work/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/huangjie/opt/anaconda3/envs/ee541_work/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/huangjie/opt/anaconda3/envs/ee541_work/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/huangjie/opt/anaconda3/envs/ee541_work/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  

0.048451151707704525 {'C': 1000}


/Users/huangjie/opt/anaconda3/envs/ee541_work/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


0.04277337012763022 {'C': 1000}

In [29]:
summary_param["svm_l1_smote"]=[{'C': 100},{'C': 1000},{'C': 100}]

In [10]:
#the best parameter for Genus label with l1-SVM smote
best_param(svm_l1_smote,y_train.iloc[:,1])

/Users/huangjie/opt/anaconda3/envs/ee541_work/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/huangjie/opt/anaconda3/envs/ee541_work/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/huangjie/opt/anaconda3/envs/ee541_work/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/huangjie/opt/anaconda3/envs/ee541_work/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/huangjie/opt/anaconda3/envs/ee541_work/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  

0.04277337012763022 {'C': 1000}


/Users/huangjie/opt/anaconda3/envs/ee541_work/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


0.0488122356003905 {'C': 100}
0.03788271160875513 {'C': 100}

In [11]:
best_param(svm_l1_smote,y_train.iloc[:,0])
best_param(svm_l1_smote,y_train.iloc[:,2])

/Users/huangjie/opt/anaconda3/envs/ee541_work/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/huangjie/opt/anaconda3/envs/ee541_work/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/huangjie/opt/anaconda3/envs/ee541_work/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/huangjie/opt/anaconda3/envs/ee541_work/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/huangjie/opt/anaconda3/envs/ee541_work/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  

0.0488122356003905 {'C': 100}


/Users/huangjie/opt/anaconda3/envs/ee541_work/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/huangjie/opt/anaconda3/envs/ee541_work/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/huangjie/opt/anaconda3/envs/ee541_work/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/huangjie/opt/anaconda3/envs/ee541_work/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/huangjie/opt/anaconda3/envs/ee541_work/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  

0.03788271160875513 {'C': 100}


In [30]:
summary_param

,svm_fixed_gamma,svm_l1,svm_l1_smote
0,"{'C': 10, 'gamma': 0.1}",{'C': 100},{'C': 100}
1,"{'C': 10, 'gamma': 0.1}",{'C': 1000},{'C': 1000}
2,"{'C': 10, 'gamma': 0.1}",{'C': 10000},{'C': 100}


In [33]:
#evaluate classifier
for i in range(3):
    print("score for label ",label.columns[i])
    print("SVM with Gaussian kernel",eva(summary_param["svm_fixed_gamma"][i]["C"],y_train.iloc[:,i],y_test.iloc[:,i]))
    print("l1-penalty SVM",eva(summary_param["svm_l1"][i]["C"],y_train.iloc[:,i],y_test.iloc[:,i]))
    print("l1-penalty SVM with SMOTE",eva_smote(summary_param["svm_l1_smote"][i]["C"],y_train.iloc[:,i],y_test.iloc[:,i]))

score for label  Family


/Users/huangjie/opt/anaconda3/envs/ee541_work/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


SVM with Gaussian kernel 0.9263547938860583


/Users/huangjie/opt/anaconda3/envs/ee541_work/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


l1-penalty SVM 0.9263547938860583


...

/Users/huangjie/opt/anaconda3/envs/ee541_work/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


SVM with Gaussian kernel 0.9615562760537286


/Users/huangjie/opt/anaconda3/envs/ee541_work/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


l1-penalty SVM 0.9615562760537286
l1-penalty SVM with SMOTE 0.9610930986567855


/Users/huangjie/opt/anaconda3/envs/ee541_work/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Conslusion: the performance of L1-penalized SVM is close to that of SVM with Gaussian kernel. However, the latter takes more effort to find the best parameter.